In [1]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = "jbhanjaya"
os.environ["KAGGLE_KEY"] = "7bdde427998742b5849efd0b498a096a"

In [2]:
!pip install -q -U keras-nlp --use-deprecated=legacy-resolver
!pip install -q -U keras>=3 --use-deprecated=legacy-resolver

In [3]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

In [4]:
import keras
import keras_nlp

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [7]:
gemma_lm.generate("How does the brain work?", max_length=64)

'How does the brain work?\n\nThe brain is the most complex organ in the human body. It is responsible for controlling all of the body’s functions, including breathing, heart rate, digestion, and more. The brain is also responsible for thinking, feeling, and making decisions.\n\nThe brain is made up'

In [8]:
!pip install datasets

In [9]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 256916
    })
})


In [11]:
!pip install jsonlines

In [12]:
import jsonlines
# Extract columns from train set
train_data = dataset['train']
descriptions = train_data['Description']
patients = train_data['Patient']
doctors = train_data['Doctor']

# Save to JSONL
with jsonlines.open('ai_medical_chatbot_train.jsonl', mode='w') as writer:
    for description, patient, doctor in zip(descriptions, patients, doctors):
        writer.write({
            'Description': description,
            'Patient': patient,
            'Doctor': doctor
        })

In [13]:
# Buka file JSONL
with jsonlines.open('ai_medical_chatbot_train.jsonl', mode='r') as reader:
    # Baca satu baris pertama
    first_line = reader.read()

# Cetak baris pertama
print(first_line)

{'Description': 'Q. What does abutment of the nerve root mean?', 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?', 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}


In [14]:
import jsonlines
import json
# Membuat list kosong untuk menyimpan data
data = []

# Buka file JSONL
with jsonlines.open('ai_medical_chatbot_train.jsonl', mode='r') as reader:
    # Membaca setiap baris dari file
    for features in reader:
        # Format teks instruksi dan tanggapan
        template = "Instruction:\n{Patient}\n\nResponse:\n{Doctor}"
        data.append(template.format(**features))

# Cetak data (10 contoh pertama)
print(data[:10])

['Instruction:\nHi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?\n\nResponse:\nHi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->', 'Instruction:\nHi doctor, I am a 22-year-old female who was diagnosed with hypothyroidism (genetic) when I was 12. Over the past five years, I have become around 50 pounds overweight and all of my attempts to lose have seemed to fail so I have given up, but my weight has stayed the same. There is so much information put there about losing weight with hypothyroidism but it all seems to conflict. I am so unsure as to what type of exercise and diet I should follow as a result but I still would like to lose weight, but most importantly have my body feel better. What can I do? I am currently on Levothyroxine, Buspar, and Bened

In [15]:
data = data[:500]

In [25]:
print(data[0])

Instruction:
Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for annular bulging and tear?

Response:
Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->


In [38]:
#tdk dipakai
'''
import json
data = []
with open("ai_medical_chatbot_train.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["Description"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{Patient}\n\nResponse:\n{Doctor}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]
'''

In [16]:
#lora enable cmn bisa dirun 1 kli
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [17]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 128
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 498s 435ms/step - loss: 2.4328 - sparse_categorical_accuracy: 0.4466


In [36]:
prompt = template.format(
    Patient="My weight is 65 kg and I am 5'4 height. It would be really helpful if you could suggest me a diet plan. ",
    Doctor="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
My weight is 65 kg and I am 5'4 height. It would be really helpful if you could suggest me a diet plan. 

Response:
Hello, I am Dr. Shweta Gupta. I am a dietician and nutritionist. Thank you for sharing the details.

I would like to know the following details:

1. Do you have diabetes or blood sugar problem?

2. Do you have any digestive problems?

3. Do you have any food allergy?

4. How many times you have food in a day?


In [41]:
prompt = template.format(
    Patient="I have a gastric pains, what should I do?",
    Doctor="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I have a gastric pains, what should I do?

Response:
Hello. I am Dr. Debjani. I have gone through your details. I understand your concern. You have mentioned that you have a burning sensation in the abdomen. It could be due to a peptic ulcer or it could be due to a gastric problem. You should consult a gastroenterologist. You may have to undergo an upper gastrointestinal endoscopy for further evaluation.


In [27]:
gemma_lm.save("gemma_lm_finetuned_v1.h5")

In [32]:
from keras.models import load_model
from keras.layers import Layer

In [35]:
gemma_lm.backbone.load_lora_weights("gemma_lm_finetuned_v1.h5")

KeyError: 'rank'